# 선보엔젤 파트너스 투자 정보 시스템

In [4]:
import json
import pandas as pd
import numpy as np
import re
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from time import time
import pickle
import os
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [5]:
from konlpy.tag import Kkma,Okt
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import NounLMatchTokenizer
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from collections import namedtuple
import multiprocessing
import logging

### loading tokenized data 

In [29]:
doc_set = pd.read_excel('C:/Users/hangy/Desktop/topics/data/doc_set.xlsx')

In [30]:
doc_set['token'] = doc_set['token'].apply(lambda x: x.replace("['","").replace("']","").split("', '"))

In [31]:
new_small = pd.read_excel('C:/Users/hangy/Desktop/topics/data/new_small_class.xlsx')

In [32]:
category_dic = {}
big = list(set(doc_set['new_class']))
for i in range(len(big)):
    temp = big[i]
    s_temp = list (set(doc_set[doc_set['new_class']==temp].new_small_class))
    category_dic[temp]=s_temp

# 2nd preprocessing

In [33]:
# noun corpus
def noun_corpus(sents):
    noun_extractor = LRNounExtractor_v2(verbose=True, extract_compound=True)
    noun_extractor.train(sents)
    nouns = noun_extractor.extract()

    noun_scores = {noun:score[0] for noun, score in nouns.items() if len(noun) > 1}
    tokenizer = NounLMatchTokenizer(noun_scores)
    corpus = [tokenizer.tokenize(sent) for sent in sents]
    return corpus

ko_stopwords = pd.read_csv('C:/Users/hangy/Desktop/topics/data/korean_stopwords.txt',encoding='utf-8-sig', engine='python')
stops = list(ko_stopwords['stopwords'])

# stopwords 제거
def stopwords (stops,corpus):
    docs=[]
    for i in range(len(corpus)):
        words=[]
        for w in corpus[i]:
            if (not w in stops) & (len(w)>1):
                words.append(w)
        docs.append(words)
    return docs

# doc2vec modeling

In [34]:
label_type = 'new_small_class'
df_train, df_test = train_test_split(doc_set, test_size=0.3,random_state=42)
print(len(df_train),len(df_test))

7138 3060


In [35]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in df_train[['token', 'new_small_class']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in df_test[['token', 'new_small_class']].values]

In [36]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [37]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=4,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [38]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

2019-10-19 16:40:12,772 : INFO : collecting all words and their counts
2019-10-19 16:40:12,774 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-10-19 16:40:14,255 : INFO : collected 46018 word types and 430 unique tags from a corpus of 7138 examples and 2238696 words
2019-10-19 16:40:14,256 : INFO : Loading a fresh vocabulary
2019-10-19 16:40:14,784 : INFO : effective_min_count=5 retains 25164 unique words (54% of original 46018, drops 20854)
2019-10-19 16:40:14,785 : INFO : effective_min_count=5 leaves 2188208 word corpus (97% of original 2238696, drops 50488)
2019-10-19 16:40:14,974 : INFO : deleting the raw counts dictionary of 46018 items
2019-10-19 16:40:14,977 : INFO : sample=0.001 downsamples 11 most-common words
2019-10-19 16:40:14,979 : INFO : downsampling leaves estimated 2184149 word corpus (99.8% of prior 2188208)
2019-10-19 16:40:15,030 : INFO : constructing a huffman tree from 25164 words
2019-10-19 16:40:15,950 : INFO : built huffman t

Doc2Vec(dbow+w,d300,n10,hs,w8,mc5,s0.001,t4)


In [32]:
# start = time()
# for epoch in range(5):
#     doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
#     doc_vectorizer.alpha -= 0.002 # decrease the learning rate
#     doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
# end = time()
# print("During Time: {}".format(end-start))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until
2019-03-28 13:44:15,382 : INFO : training model with 4 workers on 27512 vocabulary and 300 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2019-03-28 13:44:17,729 : INFO : EPOCH 1 - PROGRESS: at 0.32% examples, 3384 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:44:20,362 : INFO : EPOCH 1 - PROGRESS: at 1.89% examples, 9512 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:44:23,259 : INFO : EPOCH 1 - PROGRESS: at 2.97% examples, 10855 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:44:25,796 : INFO : EPOCH 1 - PROGRESS: at 4.63% examples, 12015 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:44:28,427 : INFO : EPOCH 1 - PROGRESS: at 6.26% examples, 12614 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:44:29,932 : INFO : EPOC

2019-03-28 13:47:05,752 : INFO : EPOCH 1 - PROGRESS: at 81.28% examples, 12090 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:08,573 : INFO : EPOCH 1 - PROGRESS: at 82.37% examples, 12061 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:10,361 : INFO : EPOCH 1 - PROGRESS: at 82.77% examples, 11995 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:12,639 : INFO : EPOCH 1 - PROGRESS: at 83.94% examples, 12005 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:14,071 : INFO : EPOCH 1 - PROGRESS: at 84.33% examples, 11965 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:15,520 : INFO : EPOCH 1 - PROGRESS: at 85.32% examples, 12029 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:17,238 : INFO : EPOCH 1 - PROGRESS: at 85.69% examples, 11970 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:19,260 : INFO : EPOCH 1 - PROGRESS: at 86.93% examples, 11997 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:47:21,001 : INFO : EPOCH 1 - PROGRESS: at 87.30% examples, 11937 words/s, in_qsize 7, out_

2019-03-28 13:49:17,546 : INFO : EPOCH 2 - PROGRESS: at 41.33% examples, 11904 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:21,089 : INFO : EPOCH 2 - PROGRESS: at 43.02% examples, 11877 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:24,467 : INFO : EPOCH 2 - PROGRESS: at 44.63% examples, 11866 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:28,860 : INFO : EPOCH 2 - PROGRESS: at 46.33% examples, 11740 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:49:30,048 : INFO : EPOCH 2 - PROGRESS: at 47.41% examples, 11893 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:49:33,475 : INFO : EPOCH 2 - PROGRESS: at 47.83% examples, 11602 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:36,320 : INFO : EPOCH 2 - PROGRESS: at 49.20% examples, 11659 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:39,167 : INFO : EPOCH 2 - PROGRESS: at 50.85% examples, 11713 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:49:42,471 : INFO : EPOCH 2 - PROGRESS: at 52.53% examples, 11721 words/s, in_qsize 7, out_

2019-03-28 13:51:49,119 : INFO : EPOCH 3 - PROGRESS: at 12.70% examples, 11083 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:51:50,576 : INFO : EPOCH 3 - PROGRESS: at 13.99% examples, 11517 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:51:52,200 : INFO : EPOCH 3 - PROGRESS: at 14.38% examples, 11240 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:51:54,046 : INFO : EPOCH 3 - PROGRESS: at 15.53% examples, 11485 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:51:56,549 : INFO : EPOCH 3 - PROGRESS: at 15.97% examples, 10972 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:51:58,101 : INFO : EPOCH 3 - PROGRESS: at 17.12% examples, 11270 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:51:59,296 : INFO : EPOCH 3 - PROGRESS: at 17.28% examples, 11120 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:52:01,257 : INFO : EPOCH 3 - PROGRESS: at 17.69% examples, 10832 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:52:02,489 : INFO : EPOCH 3 - PROGRESS: at 18.36% examples, 10983 words/s, in_qsize 8, out_

2019-03-28 13:54:28,561 : INFO : EPOCH 3 - PROGRESS: at 80.85% examples, 10850 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:31,211 : INFO : EPOCH 3 - PROGRESS: at 82.37% examples, 10903 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:32,217 : INFO : EPOCH 3 - PROGRESS: at 82.75% examples, 10896 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:54:34,093 : INFO : EPOCH 3 - PROGRESS: at 83.94% examples, 10943 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:36,775 : INFO : EPOCH 3 - PROGRESS: at 85.32% examples, 10991 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:37,955 : INFO : EPOCH 3 - PROGRESS: at 85.74% examples, 10974 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:40,079 : INFO : EPOCH 3 - PROGRESS: at 86.93% examples, 11005 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:41,213 : INFO : EPOCH 3 - PROGRESS: at 87.34% examples, 10991 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:54:43,495 : INFO : EPOCH 3 - PROGRESS: at 88.30% examples, 11011 words/s, in_qsize 7, out_

2019-03-28 13:56:48,519 : INFO : EPOCH 4 - PROGRESS: at 39.71% examples, 11461 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:56:51,758 : INFO : EPOCH 4 - PROGRESS: at 41.33% examples, 11484 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:56:53,137 : INFO : EPOCH 4 - PROGRESS: at 42.20% examples, 11526 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:56:55,272 : INFO : EPOCH 4 - PROGRESS: at 43.02% examples, 11477 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:56:56,916 : INFO : EPOCH 4 - PROGRESS: at 43.85% examples, 11482 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:56:59,998 : INFO : EPOCH 4 - PROGRESS: at 44.63% examples, 11326 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:57:01,081 : INFO : EPOCH 4 - PROGRESS: at 45.52% examples, 11399 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:57:04,179 : INFO : EPOCH 4 - PROGRESS: at 46.33% examples, 11251 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:57:05,687 : INFO : EPOCH 4 - PROGRESS: at 47.41% examples, 11370 words/s, in_qsize 8, out_

2019-03-28 13:59:19,693 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-28 13:59:19,775 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-28 13:59:20,249 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-28 13:59:20,634 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-28 13:59:20,636 : INFO : EPOCH - 4 : training on 2563578 raw words (2551096 effective words) took 241.8s, 10550 effective words/s
2019-03-28 13:59:23,597 : INFO : EPOCH 5 - PROGRESS: at 0.32% examples, 2681 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:59:27,325 : INFO : EPOCH 5 - PROGRESS: at 1.84% examples, 6934 words/s, in_qsize 7, out_qsize 0
2019-03-28 13:59:30,552 : INFO : EPOCH 5 - PROGRESS: at 2.97% examples, 8625 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:59:34,428 : INFO : EPOCH 5 - PROGRESS: at 4.63% examples, 9073 words/s, in_qsize 8, out_qsize 0
2019-03-28 13:59:38,243 : INFO : EPOCH 5 - PROGRESS: at 6.

2019-03-28 14:02:08,594 : INFO : EPOCH 5 - PROGRESS: at 71.53% examples, 10778 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:02:09,842 : INFO : EPOCH 5 - PROGRESS: at 72.36% examples, 10814 words/s, in_qsize 8, out_qsize 0
2019-03-28 14:02:12,338 : INFO : EPOCH 5 - PROGRESS: at 72.97% examples, 10770 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:02:13,776 : INFO : EPOCH 5 - PROGRESS: at 73.82% examples, 10795 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:02:17,044 : INFO : EPOCH 5 - PROGRESS: at 74.62% examples, 10706 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:02:18,518 : INFO : EPOCH 5 - PROGRESS: at 75.38% examples, 10728 words/s, in_qsize 8, out_qsize 0
2019-03-28 14:02:21,185 : INFO : EPOCH 5 - PROGRESS: at 76.16% examples, 10677 words/s, in_qsize 8, out_qsize 0
2019-03-28 14:02:22,639 : INFO : EPOCH 5 - PROGRESS: at 77.01% examples, 10698 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:02:24,184 : INFO : EPOCH 5 - PROGRESS: at 77.82% examples, 10715 words/s, in_qsize 8, out_

2019-03-28 14:04:09,168 : INFO : EPOCH 1 - PROGRESS: at 26.30% examples, 12265 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:10,876 : INFO : EPOCH 1 - PROGRESS: at 27.53% examples, 12409 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:13,410 : INFO : EPOCH 1 - PROGRESS: at 27.95% examples, 12043 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:14,844 : INFO : EPOCH 1 - PROGRESS: at 29.15% examples, 12239 words/s, in_qsize 8, out_qsize 0
2019-03-28 14:04:16,350 : INFO : EPOCH 1 - PROGRESS: at 29.42% examples, 12058 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:17,898 : INFO : EPOCH 1 - PROGRESS: at 30.45% examples, 12219 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:19,569 : INFO : EPOCH 1 - PROGRESS: at 30.85% examples, 12059 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:22,128 : INFO : EPOCH 1 - PROGRESS: at 32.33% examples, 12180 words/s, in_qsize 7, out_qsize 0
2019-03-28 14:04:24,726 : INFO : EPOCH 1 - PROGRESS: at 33.84% examples, 12284 words/s, in_qsize 7, out_

KeyboardInterrupt: 

In [ ]:
# model_name = 'Doc2vec1.model'
# doc_vectorizer.save(model_name)

In [39]:
# loading Doc2vec model
model_name = 'Doc2vec1.model'
doc_vectorizer = Doc2Vec.load(model_name)

2019-10-19 16:40:21,988 : INFO : loading Doc2Vec object from Doc2vec1.model
2019-10-19 16:40:23,642 : INFO : loading vocabulary recursively from Doc2vec1.model.vocabulary.* with mmap=None
2019-10-19 16:40:23,643 : INFO : loading trainables recursively from Doc2vec1.model.trainables.* with mmap=None
2019-10-19 16:40:23,644 : INFO : loading wv recursively from Doc2vec1.model.wv.* with mmap=None
2019-10-19 16:40:23,646 : INFO : loading docvecs recursively from Doc2vec1.model.docvecs.* with mmap=None
2019-10-19 16:40:23,647 : INFO : loaded Doc2vec1.model


In [40]:
print(doc_vectorizer.wv.most_similar('LNG'))

2019-10-19 16:40:24,349 : INFO : precomputing L2-norms of word weight vectors


[('벙커링', 0.6494147181510925), ('연료추진선', 0.6215590238571167), ('운반선', 0.5957974195480347), ('LNG연료공급', 0.5679702758789062), ('LNG연료추진선', 0.567599892616272), ('VLCC', 0.563372790813446), ('LNG벙커링', 0.5483577847480774), ('추진선박', 0.5353018641471863), ('LNG연료', 0.5342037677764893), ('추진선', 0.5339316725730896)]


In [41]:
print(doc_vectorizer.wv.most_similar(positive=['선박', '스마트'], negative=['금속']))

[('추진시스템', 0.3174276351928711), ('추진계통', 0.31573396921157837), ('해양사고', 0.30571871995925903), ('전류고정날개', 0.3017979860305786), ('성능해석기술', 0.29743826389312744), ('시설재배', 0.2963535189628601), ('예인선', 0.2923802137374878), ('기존선박', 0.29178160429000854), ('드릴쉽', 0.288592666387558), ('추진선', 0.2880726754665375)]


In [42]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]
X_test  = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
y_test  = [doc.tags for doc in tagged_test_docs]

## classification modeling

In [43]:
doc_set.head()

,year,big_class,small_class,content,token,big_calss_re,new_class,new_small_class
0,2017,항공우주,차세대 드론 개발 기술,개요 가 정의 및 필요성 차세대 드론 개발 기술은 무선전파로 조정할 ...,"[개요, 정의, 필요성, 차세대, 드론, 무선전파, 조정, 무인항공기, 드론, 위하...",항공우주,항공우주,차세대 드론 개발 기술
1,2017,항공우주,차세대 드론 개발 기술,나 범위 제품분류 관점 드론은 사용 목적 탑재되는 임무장비에 따라...,"[범위, 제품분류, 관점, 드론, 목적, 탑재, 임무장비, 분류, 드론, 구성, 핵...",항공우주,항공우주,차세대 드론 개발 기술
2,2017,항공우주,차세대 드론 개발 기술,외부환경 분석 가 산업환경 분석 산업의 특징 드론 산업은 차...,"[외부환경, 산업환경, 특징, 드론, 집약체, AI, 자율지능, 프린팅, 연료전지,...",항공우주,항공우주,차세대 드론 개발 기술
3,2017,항공우주,차세대 드론 개발 기술,나 시장환경 분석 세계시장 미국 방위산업 전문 컨설팅 업체인 틸그룹...,"[시장환경, 세계시장, 방위산업, 전문, 컨설팅, 업체, Group, 세계, 드론,...",항공우주,항공우주,차세대 드론 개발 기술
4,2017,항공우주,차세대 드론 개발 기술,무역현황 무역현황은 드론과 관련된 무역현황으로 살펴보았으며 ‘ 년...,"[무역현황, 무역현황, 드론, 무역현황, 이후, 수출량, 줄어, 수출량, 늘어, 추...",항공우주,항공우주,차세대 드론 개발 기술


In [44]:
category_dic = {}
big = list(set(doc_set['new_class']))
for i in range(len(big)):
    temp = big[i]
    s_temp = list (set(doc_set[doc_set['new_class']==temp].new_small_class))
    category_dic[temp]=s_temp

In [45]:
label_type = 'new_small_class'
df_train, df_test = train_test_split(doc_set, test_size=0.3,random_state=42)
print(len(df_train),len(df_test))

7138 3060


In [46]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in df_train[['token', 'new_class']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in df_test[['token', 'new_class']].values]
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]
X_test  = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
y_test  = [doc.tags for doc in tagged_test_docs]

In [47]:
tagged_train_docs2 = [TaggedDocument(d, c) for d, c in df_train[['token', 'new_small_class']].values]
tagged_test_docs2 = [TaggedDocument(d, c) for d, c in df_test[['token', 'new_small_class']].values]
y_train2 = [doc.tags for doc in tagged_train_docs2]
y_test2  = [doc.tags for doc in tagged_test_docs2]

In [48]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(1000,), max_iter=10, alpha=1e-4, solver='sgd',
                        verbose=10, tol=1e-4, random_state=123,learning_rate_init=.1)

In [49]:
mlp_clf2 = MLPClassifier(
    hidden_layer_sizes=(1000,),
    max_iter=20,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=123,
    learning_rate_init=.1
)

In [50]:
mlp_clf.fit(X_train, y_train)

y_pred = mlp_clf.predict(X_test)
print("1class 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))
y_pred_prob = mlp_clf.predict_proba(X_test)
L = np.argsort(-y_pred_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {mlp_clf.classes_[i]: i for i in range(len(mlp_clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []
score = []
for i in range(len(y_test)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    third = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(third)]])
    if y_test[i] in label :
        score.append(1)
    else : 
        score.append(0)
    dd.append({'y_test':y_test[i], 'first':label[0], 'second': label[1],'third': label[2]})
acc = sum(score)/len(y_test)
print("3class 정확도:", acc)

Iteration 1, loss = 2.54369531
Iteration 2, loss = 0.80737445
Iteration 3, loss = 0.41704808
Iteration 4, loss = 0.28498373
Iteration 5, loss = 0.21531149
Iteration 6, loss = 0.16715184
Iteration 7, loss = 0.13041851
Iteration 8, loss = 0.10757112
Iteration 9, loss = 0.08695456
Iteration 10, loss = 0.07156690


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1class 정확도: 0.853
3class 정확도: 0.9640522875816994


In [51]:
mlp_clf2.fit(X_train, y_train2)
y_pred = mlp_clf2.predict(X_test)
y_pred_prob = mlp_clf2.predict_proba(X_test)
L = np.argsort(-y_pred_prob, axis=1)
two_pred = L[:,0:3]
class_dic = {mlp_clf2.classes_[i]: i for i in range(len(mlp_clf2.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values())

Iteration 1, loss = 5.89942825
Iteration 2, loss = 4.74950241
Iteration 3, loss = 3.10452287
Iteration 4, loss = 1.75152436
Iteration 5, loss = 1.00247582
Iteration 6, loss = 0.62821776
Iteration 7, loss = 0.43654928
Iteration 8, loss = 0.31564458
Iteration 9, loss = 0.23746317
Iteration 10, loss = 0.18762772
Iteration 11, loss = 0.14829516
Iteration 12, loss = 0.12536452
Iteration 13, loss = 0.10329625
Iteration 14, loss = 0.08991689
Iteration 15, loss = 0.07822615
Iteration 16, loss = 0.06910897
Iteration 17, loss = 0.06293344
Iteration 18, loss = 0.05641129
Iteration 19, loss = 0.05240951
Iteration 20, loss = 0.04863279


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [52]:
second_result = []
score = []
for i in range(len(y_test2)):
    tt = category_dic[dd[i]['first']]+category_dic[dd[i]['second']]
    tt = [x for x in tt if x in key_list]
    ttt = list({ your_key: class_dic[your_key] for your_key in tt }.values())
    tttt = [x for x in L[i] if x in ttt]
    first = tttt[0]
    second = tttt[1]
    third = tttt[2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(third)]])
    if y_test2[i] in label :
        score.append(1)
    else : 
        score.append(0)
    second_result.append({'y_test':y_test2[i], 'first':label[0], 'second': label[1],'third': label[2]})
acc = sum(score)/len(second_result)

In [53]:
acc

0.8797385620915033

### 1. logistic regression

In [29]:
clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("1class 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))
y_pred_prob = clf.predict_proba(X_test)
L = np.argsort(-y_pred_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {clf.classes_[i]: i for i in range(len(clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []
score = []
for i in range(len(y_test)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    third = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(third)]])
    if y_test[i] in label :
        score.append(1)
    else : 
        score.append(0)
    dd.append({'y_test':y_test[i], 'first':label[0], 'second': label[1],'third': label[2]})
acc = sum(score)/len(y_test)
print("3class 정확도:", acc)

3class 정확도: 0.8209150326797385


### 2. Neural network

In [133]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(1000,),
    max_iter=15,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=123,
    learning_rate_init=.1
)
mlp_clf.fit(X_train, y_train)

y_pred = mlp_clf.predict(X_test)
print("1class 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))
y_pred_prob = mlp_clf.predict_proba(X_test)
L = np.argsort(-y_pred_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {mlp_clf.classes_[i]: i for i in range(len(mlp_clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []
score = []
for i in range(len(y_test)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    third = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(third)]])
    if y_test[i] in label :
        score.append(1)
    else : 
        score.append(0)
    dd.append({'y_test':y_test[i], 'first':label[0], 'second': label[1],'third': label[2]})
acc = sum(score)/len(y_test)
print("3class 정확도:", acc)

Iteration 1, loss = 5.91254410
Iteration 2, loss = 4.75403352
Iteration 3, loss = 3.11628984
Iteration 4, loss = 1.75860231
Iteration 5, loss = 1.01270443
Iteration 6, loss = 0.63180073
Iteration 7, loss = 0.43230865
Iteration 8, loss = 0.31561289
Iteration 9, loss = 0.23740707
Iteration 10, loss = 0.18648184
Iteration 11, loss = 0.15067683
Iteration 12, loss = 0.12489761
Iteration 13, loss = 0.10358679
Iteration 14, loss = 0.08960562
Iteration 15, loss = 0.07758507
Iteration 16, loss = 0.06901886
Iteration 17, loss = 0.06292978
Iteration 18, loss = 0.05487183
Iteration 19, loss = 0.05163544
Iteration 20, loss = 0.04740343


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


1class 정확도: 0.738
3class 정확도: 0.9035317200784827


## calssification start-up 

In [54]:
company_list = pd.read_csv('C:/Users/hangy/Desktop/topics/data/company_list.csv',encoding='utf-8-sig',engine='python')
company_list = company_list.replace(np.nan, '', regex=True)
company_list.head()

,category,company,invest_num,overview,sub_info,naver_category,naver_detail,intro
0,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'e-c...",쿠팡,3,"""쿠팡은 세계에서 가장 빠르고 크게 성장하는 E-commerce 기업입니다. 우리는...","['설립일: 2010-07-01\xa0/\xa09년차', '...","이커머스, 쇼핑몰, 마트, 패션의류/잡화, 뷰티, 여행/레저, 할인.","{'기업명': '쿠팡(주)', '기업구분': '중견기업', '대표자': '범석김',...",
1,"['마케팅', '스타트업', 'startup']",KOTRA 수출창업지원팀,1,국내 스타트업 기업의 글로벌 시장 진출 지원 \n\n\n\n,"['투자유치: 누적 6,000억 원 이상 상세보...",None,None,
2,['온라인게임'],넷마블게임즈,1,none,"['투자유치: 누적 5,300억 원 이상 상세보...","2011년 11월 설립된 동사는 개발 스튜디오에서 게임을 직접 개발하고, PC/모바...","{'기업명': '넷마블(주)', '기업구분': '중견기업, 코스피 상장', '대표자...",\n 씨제이넷마블(주)와 씨제이게임즈(주)가 넷마블게임즈(주)로...
3,"['모바일광고', '모바일광고플랫폼', '애드테크', '애드네트워크', 'DSP',...",Avazu Korea,1,글로벌 유니콘 스타트업이 한국에 진출합니다! About usDotC United G...,"['구성원: 501-1000명 상세보기', '투...",None,None,
4,['통신'],SK브로드밴드,2,none,"['설립일: 1997-09-26\xa0/\xa022년차', ...",None,None,\n 에스케이브로드밴드(주)는 케이블 TV서비스 및 초고속 인터...


#### preprocessing

In [44]:
document = []
for i in range(len(company_list)):
    if ('기업 소개가 없습니다' in company_list['overview'][i]) or (company_list['overview'][i] == 'none'): 
        document.append(company_list['category'][i])
    else :
        document.append(company_list['category'][i] + company_list['overview'][i])
        
document = []
for i in range(len(company_list)):
    document.append(company_list['category'][i] + company_list['overview'][i]+company_list['naver_category'][i]+company_list['intro'][i])
    
for i in range(len(document)):
    document[i] = re.sub('[\n\uf000\x0c()/▪,.ㆍ\[\]0-9\'●※\"▷▶:\};&\-><%|🌟✔️]',' ', document[i])
    
company_data = pd.DataFrame({'company':company_list['company'],'category':company_list['category'],
                             'document':document,'lable':['']*len(document)})

company_corpus = noun_corpus(company_data['document'])
company_docs = stopwords(stops, company_corpus)
company_data['token'] = company_docs

stops = list(ko_stopwords['stopwords'])
company_docs = stopwords(stops, company_docs)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 58194 from 2454 sents. mem=1.168 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=244906, mem=1.266 Gb
[Noun Extractor] batch prediction was completed for 19105 words
[Noun Extractor] checked compounds. discovered 8021 compounds
[Noun Extractor] postprocessing detaching_features : 14425 -> 12844
[Noun Extractor] postprocessing ignore_features : 12844 -> 12804
[Noun Extractor] postprocessing ignore_NJ : 12804 -> 12769
[Noun Extractor] 12769 nouns (8021 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.295 Gb                    
[Noun Extractor] 66.75 % eojeols are covered


In [45]:
stops = list(ko_stopwords['stopwords'])
company_docs = stopwords(stops, company_docs)

company_data['token'] = company_docs
company_tokens = [ t for d in company_data['token'] for t in d]
company_text = nltk.Text(company_tokens, name='NMSC')
company_fdist = company_text.vocab()

company_fdist = pd.DataFrame.from_dict(company_fdist, orient='index')
company_fdist.columns = ['frequency']
company_fdist['term'] = list(company_fdist.index)
company_fdist = company_fdist.reset_index(drop=True)
company_fdist = company_fdist.sort_values(["frequency"], ascending=[False])
company_fdist = company_fdist.reset_index(drop=True)

In [46]:
# zif'slow 
zif_list = pd.DataFrame(company_fdist[(company_fdist['frequency'] >500) | (company_fdist['frequency'] <= 2)]['term'])
zif_list.columns = ['stopwords']
zif_list = zif_list.reset_index(drop=True)

stops = list(zif_list['stopwords'])
company_docs2 = stopwords(stops, company_docs)
company_data['token'] = company_docs2

In [47]:
token_len =[]
for i in range(len(company_data)):
    token_len.append(len(company_data['token'][i]))
company_data['token_len'] = token_len

In [48]:
sub_company = company_data[company_data['token_len']>41].reset_index(drop=True)

In [49]:
sub_company.head()

,company,category,document,lable,token,token_len
0,쿠팡,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'e-c...",모바일서비스 웹서비스 마케팅 소프트웨어 데이터 e c...,,"[commerce, 소셜커머스, 배송, 유통업, 쿠팡, 세계, commerce, 쿠...",67
1,서울반도체,['제조업'],제조업 none일반조명 IT 자동차 UV 등 광범위한 분야에 적용되는 L...,,"[제조업, IT, 자동차, 광범위, 분야, 적용, LED, 제품, 연구개발, 생산,...",90
2,롯데하이마트,"['유통', '전자제품']",유통 전자제품 none가전제품 도 소매업을 영위할 목적으로 년 ...,,"[유통, 전자제품, 소매, 영위, 목적, 물류센터, 운영, 있음, 가전제품, 전자제...",54
3,옐로모바일,"['모바일서비스', '웹서비스', '소프트웨어', '모바일게임', '소셜커머스', ...",모바일서비스 웹서비스 소프트웨어 모바일게임 소셜커머스 ...,,"[모바일게임, 소셜커머스, 오픈마켓, 옐로모바일, 옐로모바일, 먹고, 일상, 라이프...",44
4,미미박스,"['Confluence', 'JIRA', 'Google Apps', 'Slack',...",Confluence JIRA Google Apps Slack ...,,"[Confluence, commerce, io, 안드로이드, 개발자, 뷰티스타트업,...",99


In [50]:
tagged_company_docs = [TaggedDocument(d, c) for d, c in sub_company[['token', 'lable']].values]
X_company = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_company_docs]
y_company = [doc.tags for doc in tagged_company_docs]

#### fitting model

In [51]:
y_company_pred = mlp_clf.predict(X_company)
y_company_prob = mlp_clf.predict_proba(X_company)
L = np.argsort(-y_company_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {clf.classes_[i]: i for i in range(len(clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []

for i in range(len(y_company_prob)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    thrid = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(thrid)]])
    dd.append({'company':sub_company['company'][i],'origianl_label':sub_company['category'][i],
               'predicted_label1':label[0], 'predicted_label2': label[1], 'predicted_label3': label[2]})

company_pred_df = pd.DataFrame(dd,columns = ['company','origianl_label','predicted_label1','predicted_label2','predicted_label3'])

In [239]:
#company_pred_df.to_excel('C:/Users/hangy/Desktop/company_predict_result.xlsx',sheet_name='Sheet1')

In [53]:
company_pred_df.head(10)

,company,origianl_label,predicted_label1,predicted_label2,predicted_label3
0,쿠팡,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'e-c...",O2O/지식서비스,스마트미디어기기,디지털콘텐츠·디자인
1,서울반도체,['제조업'],LED/광,홈어플라이언스,전자부품
2,롯데하이마트,"['유통', '전자제품']",O2O/지식서비스,홈어플라이언스,클라우드
3,옐로모바일,"['모바일서비스', '웹서비스', '소프트웨어', '모바일게임', '소셜커머스', ...",블록체인,디지털콘텐츠·디자인,O2O/지식서비스
4,미미박스,"['Confluence', 'JIRA', 'Google Apps', 'Slack',...",O2O/지식서비스,디지털콘텐츠·디자인,AI/빅데이터
5,쏘카,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'O2O...",블록체인,AI/빅데이터,O2O/지식서비스
6,포도트리,"['Confluence', 'Google Apps', 'Slack', 'Trello...",O2O/지식서비스,디지털콘텐츠·디자인,홈어플라이언스
7,데일리금융그룹,"['IT서비스', '금융', '핀테크', 'Fintech', 'finance', '...",핀테크,블록체인,O2O/지식서비스
8,클룩,"['웹서비스', '마케팅', '플랫폼', '스타트업', '여행', '액티비티']",에너지,O2O/지식서비스,자율주행차
9,야놀자,"['모바일서비스', '웹서비스', '마케팅', '소프트웨어', '데이터', 'IT서...",O2O/지식서비스,스마트미디어기기,AI/빅데이터


## classification for TIPS 

In [389]:
tips_list = pd.read_csv('C:/Users/hangy/Desktop/topics/data/tips_list.csv',encoding='utf-8-sig',engine='python')

In [390]:
tips_news = pd.read_excel('C:/Users/hangy/Desktop/topics/data/tips_list_news.xlsx')

In [391]:
del tips_list['ceo']
del tips_list['date']
del tips_list['table']

In [392]:
tips_list.head()

,company,doc,information,category
0,(주)시옷플랫폼,시옷플랫폼은 여러 종류의 영상센서(열영상센서와 이미지센서)와 MEMS 센서를 결합한...,"{'사업분야': 'IoT(사물인터넷)', '설립일': '2010. 8. 8', '대...",IoT(사물인터넷)
1,큐키,스마트폰을 사용하게 되면서 겪는 문제는 문자입력에 오타 입력이 많다는 점을 착안하여...,"{'사업분야': '모바일IT', '설립일': '2013.07.15', '대표자': ...",모바일IT
2,(주)위브랩,위브랩은 다음(Daum) 초창기부터 검색엔진 개발과 관리를 이끌어온 핵심인력들로 구...,"{'사업분야': 'IT S/W', '설립일': '2013.08.06', '대표자':...",IT S/W
3,엔트리움(주),현재 디스플레이용 비등방성 전도성 집착필름(ACF)에 사용되는 도전성 입자는(약 2...,"{'사업분야': '기계·소재', '설립일': '2013.02.01', '대표자': ...",기계·소재
4,프라센,"프라센은 KAIST 기계공학, 전자공학 연구원 및 전 삼성전자 소프트웨어 엔지니어가...","{'사업분야': '바이오의료', '설립일': '2010.03.08', '대표자': ...",바이오의료


#### preprocessing

In [393]:
filter_idx = []
for i in range(len(tips_news)) : 
    comapny = tips_news['company'][i]
    company_name = comapny.replace("(주)","")
    if company_name in tips_news['title'][i] :
        filter_idx.append(i)

In [394]:
filter_news = tips_news[tips_news.index.isin(filter_idx)].reset_index(drop=True)

In [395]:
filter_news = filter_news.drop_duplicates(subset='company', keep="first").reset_index(drop=True)

In [396]:
sub_tips_list = pd.merge(tips_list, filter_news)

In [397]:
sub_tips_list.head()

,company,doc,information,category,title,press,link,date,content
0,큐키,스마트폰을 사용하게 되면서 겪는 문제는 문자입력에 오타 입력이 많다는 점을 착안하여...,"{'사업분야': '모바일IT', '설립일': '2013.07.15', '대표자': ...",모바일IT,김민철 큐키 대표,한경비즈니스,https://news.naver.com/main/read.nhn?mode=LSD&...,2016-04-26 18:03,대한민국 스타트업 100인김민철 큐키 대표 스마트폰 오타 수정 어렵지 않아요 ...
1,(주)위브랩,위브랩은 다음(Daum) 초창기부터 검색엔진 개발과 관리를 이끌어온 핵심인력들로 구...,"{'사업분야': 'IT S/W', '설립일': '2013.08.06', '대표자':...",IT S/W,[2020유니콘]<19>위브랩,전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2016-10-05 17:00,0302016100586215520161004141829624000299201...
2,엔트리움(주),현재 디스플레이용 비등방성 전도성 집착필름(ACF)에 사용되는 도전성 입자는(약 2...,"{'사업분야': '기계·소재', '설립일': '2013.02.01', '대표자': ...",기계·소재,"[나노 강소기업을 가다]<4>엔트리움, 신개념 전자파 차폐 소재 본격 양산…올해 '...",전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-02-19 17:02,엔트리움이 하이닉스와 협업해 개발한 스프레이 방식 전자파 차폐 소재를 올해 본격 양...
3,비트패킹컴퍼니,"비트패킹컴퍼니는 음악을 무료로 들을 수 있는 모바일 전용 스트리밍 라디오 앱, ‘비...","{'사업분야': '모바일IT', '설립일': '2013.4.24', '대표자': '...",모바일IT,"비트패킹컴퍼니 결국 서비스 종료...""추가 투자 유치 실패로 서비스 지속 불가""",조선비즈,https://news.naver.com/main/read.nhn?mode=LSD&...,2016-12-01 11:40,음악 스트리밍 업체 비트패킹컴퍼니가 지난달 30일 서비스를 종료했다 1년 넘게 추가...
4,(주)리브스메드,"기존의 복강경 수술 기구는 관절이 없는 일자형으로서, 수술 동작의 수행이 쉽지 않고...","{'사업분야': '바이오의료', '설립일': '2011.6.20', '대표자': '...",바이오의료,"리브스메드, ‘복강경 기구 국산화’ 복지부 장관상 수상",헤럴드경제,https://news.naver.com/main/read.nhn?mode=LSD&...,2018-11-06 15:01,0162018110620181106000903020181106150106592...


In [433]:
#sub_tips_list.to_excel('C:/Users/hangy/Desktop/sub_tips_list.xlsx',sheet_name='Sheet1')

In [398]:
tips_document = []
for i in range(len(sub_tips_list)):
    tips_document.append(str(sub_tips_list['doc'][i])+str(sub_tips_list['content'][i]))
    
for i in range(len(tips_document)):
    tips_document[i] = re.sub('[\n\uf000\x0c()/▪,.ㆍ\[\]0-9\'●※\"▷▶:\};&\-><%|🌟✔️]',' ', tips_document[i])
    
tips_data = pd.DataFrame({'company':sub_tips_list['company'],'category': sub_tips_list['category'],
                          'document':tips_document,'lable':['']*len(tips_document)})

tips_corpus = noun_corpus(tips_data['document'])
tips_docs = stopwords(stops, tips_corpus)
tips_data['token'] = tips_docs

stops = list(ko_stopwords['stopwords'])
tips_docs = stopwords(stops, tips_docs)
tips_data['token'] = tips_docs

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 32996 from 303 sents. mem=2.337 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=94241, mem=2.374 Gb
[Noun Extractor] batch prediction was completed for 13233 words
[Noun Extractor] checked compounds. discovered 3065 compounds
[Noun Extractor] postprocessing detaching_features : 7510 -> 7110
[Noun Extractor] postprocessing ignore_features : 7110 -> 7078
[Noun Extractor] postprocessing ignore_NJ : 7078 -> 7069
[Noun Extractor] 7069 nouns (3065 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.384 Gb                    
[Noun Extractor] 73.83 % eojeols are covered


In [399]:
tips_tokens = [ t for d in tips_data['token'] for t in d]
tips_text = nltk.Text(tips_tokens, name='NMSC')
tips_fdist = tips_text.vocab()

tips_fdist = pd.DataFrame.from_dict(tips_fdist, orient='index')
tips_fdist.columns = ['frequency']
tips_fdist['term'] = list(tips_fdist.index)
tips_fdist = tips_fdist.reset_index(drop=True)
tips_fdist = tips_fdist.sort_values(["frequency"], ascending=[False])
tips_fdist = tips_fdist.reset_index(drop=True)

In [400]:
# zif'slow 
zif_list = pd.DataFrame(tips_fdist[(tips_fdist['frequency'] >200) | (tips_fdist['frequency'] <= 2)]['term'])
zif_list.columns = ['stopwords']
zif_list = zif_list.reset_index(drop=True)

stops = list(zif_list['stopwords'])
tips_docs2 = stopwords(stops, tips_docs)
tips_data['token'] = tips_docs2

In [401]:
token_len =[]
for i in range(len(tips_data)):
    token_len.append(len(tips_data['token'][i]))
tips_data['token_len'] = token_len

In [402]:
tagged_tips_docs = [TaggedDocument(d, c) for d, c in tips_data[['token', 'lable']].values]
X_tips = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_tips_docs]
y_tips = [doc.tags for doc in tagged_tips_docs]

# 소분류

In [415]:
y_company_pred = mlp_clf.predict(X_tips)
y_company_prob = mlp_clf.predict_proba(X_tips)
L = np.argsort(-y_company_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {mlp_clf.classes_[i]: i for i in range(len(mlp_clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []

for i in range(len(y_company_prob)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    thrid = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(thrid)]])
    dd.append({'company':tips_data['company'][i],'origianl_label':tips_data['category'][i],
               'predicted_label1':label[0], 'predicted_label2': label[1], 'predicted_label3': label[2]})

#company_pred_df = pd.DataFrame(dd,columns = ['company','origianl_label','predicted_label1','predicted_label2','predicted_label3'])

In [416]:
y_company_pred = mlp_clf2.predict(X_tips)
y_company_prob = mlp_clf2.predict_proba(X_tips)
L = np.argsort(-y_company_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {mlp_clf2.classes_[i]: i for i in range(len(mlp_clf2.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

In [423]:
second_result = []
score = []
for i in range(len(X_tips)):
    tt = category_dic[dd[i]['predicted_label1']] + category_dic[dd[i]['predicted_label2']]
    tt = [x for x in tt if x in key_list]
    ttt = list({ your_key: class_dic[your_key] for your_key in tt }.values())
    tttt = [x for x in L[i] if x in ttt]
    first = tttt[0]
    second = tttt[1]
    third = tttt[2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(third)]])
    if y_test2[i] in label :
        score.append(1)
    else : 
        score.append(0)
    #second_result.append({'y_test':y_test2[i], 'first':label[0], 'second': label[1],'third': label[2]})
    second_result.append({'company':tips_data['company'][i],'origianl_label':tips_data['category'][i],
           'big_predict1' :dd[i]['predicted_label1'],'big_predict2' :dd[i]['predicted_label2'],
           'predicted_label1':label[0], 'predicted_label2': label[1], 'predicted_label3': label[2]})
acc = sum(score)/len(second_result)

In [424]:
pd.DataFrame(second_result)

,big_predict1,big_predict2,company,origianl_label,predicted_label1,predicted_label2,predicted_label3
0,AI/빅데이터,O2O/지식서비스,큐키,모바일IT,텍스트 기반 시각 데이터 이해 및 검색 서비스,인공지능 플랫폼,바이오 시큐리티 금융서비스
1,AI/빅데이터,디지털콘텐츠·디자인,(주)위브랩,IT S/W,인공지능 플랫폼,빅데이터 분석 및 시각화 플랫폼,영상처리 시스템
2,지능형반도체,스마트팩토리,엔트리움(주),기계·소재,SoC 부품,전력반도체소자,초저전압 고신뢰성 뉴로모픽 엔진탑재 마이크로컨트롤러
3,O2O/지식서비스,AI/빅데이터,비트패킹컴퍼니,모바일IT,개인맞춤형 스마트러닝,음성인식 SW,데이터 보안 및 비식별화
4,의료서비스·기기,LED/광,(주)리브스메드,바이오의료,"치료, 수술용 기구 및 기기",재활 및 보조기기,신광원기반 일반조명시스템
5,AI/빅데이터,핀테크,(주)유저해빗,모바일IT,빅데이터 분석 및 시각화 플랫폼,텍스트 기반 시각 데이터 이해 및 검색 서비스,음성인식 SW
6,O2O/지식서비스,디지털콘텐츠·디자인,(주)크레스프리,IoT(사물인터넷),공공 시설물 디자인 기술,스마트 완구,개인맞춤형 스마트러닝
7,AI/빅데이터,O2O/지식서비스,(주)다노,IT S/W,개인맞춤형 스마트러닝,데이터 보안 및 비식별화,음성인식 SW
8,블록체인,O2O/지식서비스,(주)라이클,모바일IT,블록체인 기반 온라인 쇼핑몰,O2O 서비스 중개 플랫폼,개인맞춤형 스마트러닝
9,디지털콘텐츠·디자인,O2O/지식서비스,(주)브레이브팝스컴퍼니,IT S/W,에듀테크 콘텐츠,스마트 완구,개인맞춤형 스마트러닝


In [427]:
#pd.DataFrame(second_result).to_excel('C:/Users/hangy/Desktop/tips_small_predict_result.xlsx',sheet_name='Sheet1')

#### fitting model

In [48]:
y_company_pred = mlp_clf.predict(X_tips)
y_company_prob = mlp_clf.predict_proba(X_tips)
L = np.argsort(-y_company_prob, axis=1)
two_pred = L[:,0:3]

class_dic = {mlp_clf.classes_[i]: i for i in range(len(mlp_clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dd = []

for i in range(len(y_company_prob)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    thrid = two_pred[i][2]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)],key_list[val_list.index(thrid)]])
    dd.append({'company':tips_data['company'][i],'origianl_label':tips_data['category'][i],
               'predicted_label1':label[0], 'predicted_label2': label[1], 'predicted_label3': label[2]})

company_pred_df = pd.DataFrame(dd,columns = ['company','origianl_label','predicted_label1','predicted_label2','predicted_label3'])

In [49]:
company_pred_df.origianl_label.value_counts()

정보통신          121
바이오의료          67
IT S/W         25
기계·소재          23
전기전자           22
모바일IT          13
IT 서비스         10
기타              6
화학              3
기타서비스           3
ICT제조           2
IoT(사물인터넷)      2
IT H/W          2
반도체             1
정보기기·통신장비       1
IT부품            1
에너지·자원          1
Name: origianl_label, dtype: int64

In [235]:
#company_pred_df.to_excel('C:/Users/hangy/Desktop/tips_predict_result.xlsx',sheet_name='Sheet1')

## calssification for 교수 개발 뉴스

In [105]:
professor=pd.read_csv('C:/Users/hangy/Desktop/professor.csv')

In [106]:
professor_list = []
# 텍스트를 가지고 있는 리스트
for i in list(professor['content']):
    # 숫자 및 특수문자 제거.
    t = re.sub('[\d\s0-9]',' ',str(i)).strip()
    t = re.sub('[=+,#/\?:^$.@*\"※~&%ㆍ·⌬◎◳▢▪!』․\\‘|\(\)\[\]\<\>`\'…》→’“”;●•]', ' ', t)
    t = re.sub('\xad', ' ', t)
    t = re.sub('  ', ' ', t)
    t = re.sub('  ', ' ', t)
    t = re.sub('  ', ' ', t)
    t = re.sub('및', '', t)
    professor_list.append(t)

In [107]:
professor_corpus=noun_corpus(professor_list)
#stopwords 제거
stops = list(ko_stopwords['stopwords'])
docs=[]
for i in range(len(professor_corpus)):
    words=[]
    for w in professor_corpus[i]:
        if (not w in stops) & (len(w)>1):
            words.append(w)
    docs.append(words)
professor['token']=docs

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 28004 from 470 sents. mem=0.990 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=97886, mem=0.996 Gb
[Noun Extractor] batch prediction was completed for 10688 words
[Noun Extractor] checked compounds. discovered 2949 compounds
[Noun Extractor] postprocessing detaching_features : 6722 -> 6454
[Noun Extractor] postprocessing ignore_features : 6454 -> 6426
[Noun Extractor] postprocessing ignore_NJ : 6426 -> 6422
[Noun Extractor] 6422 nouns (2949 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.997 Gb                    
[Noun Extractor] 73.30 % eojeols are covered


In [109]:
tokens = [ t for d in professor['token'] for t in d]
text = nltk.Text(tokens, name='NMSC')
fdist = text.vocab()
df_fdist = pd.DataFrame.from_dict(fdist, orient='index')
df_fdist.columns = ['frequency']
df_fdist['term'] = list(df_fdist.index)
df_fdist = df_fdist.reset_index(drop=True)
df_fdist = df_fdist.sort_values(["frequency"], ascending=[False])
df_fdist = df_fdist.reset_index(drop=True)

In [110]:
tokens = [ t for d in professor['token'] for t in d]
text   = nltk.Text(tokens, name='NMSC')
print('token 개수           : ',len(df_fdist))
print('상위 횟수 token 개수 : ',len(df_fdist[df_fdist['frequency']>300]))
print('1회 횟수 token 개수  : ',len(df_fdist[df_fdist['frequency']<=10]))

token 개수           :  5800
상위 횟수 token 개수 :  10
1회 횟수 token 개수  :  4663


In [112]:
zif_list = pd.DataFrame(df_fdist[(df_fdist['frequency'] >250) | (df_fdist['frequency'] <= 2)]['term'])
zif_list.columns = ['stopwords']
zif_list = zif_list.reset_index(drop=True)

# zif'slow 
stops = list(zif_list['stopwords'])
def stopwords (stops,corpus):
    docs=[]
    for i in range(len(corpus)):
        words=[]
        for w in corpus[i]:
            if (not w in stops) & (len(w)>1):
                words.append(w)
        docs.append(words)
    return docs
docs = stopwords(stops, professor['token'])
professor['token'] = docs

In [114]:
stops = ['사진제공','박사','전남대','사진','제공','저자','전기전자컴퓨터공학부','기자','선준미디어','교재','밝혔다새로','연구사업','공동연구','석사과정','개최','시앙스포','위치','배출','어렵다','교수연구팀','고려대학교','부산대', '제시','GIST','코엑스','네이버','채널','구독','미국','국내','논문','게재','학술지','연구원','뉴시스통신','연구결과','제출','관련','때문','뉴시스','대학원','대통령','프랑스','합격','장관','정무영','이전','주요','인사','자랑','한계','진출','눈길','세계','뉴스','기자','어려움','대학원생','국내외','기관','박사과정','연구진','졸업생','UNIST','광운대학교','총장','있었다','예상된다','대학']
docs = stopwords(stops, professor['token'])
professor['token'] = docs

In [116]:
professor['class']='None'
pro_train_docs = [TaggedDocument(d,c) for d,c in professor[['token', 'class']].values]
pro_train = [doc_vectorizer.infer_vector(doc.words) for doc in pro_train_docs]

In [119]:
#pro_y = mlp_clf.predict(pro_train)

pro_y = mlp_clf.predict(pro_train)
#print("1class 정확도: {:.3f}".format(accuracy_score(pro_y, pro_train)))
pro_y_pred_prob = mlp_clf.predict_proba(pro_train)
L = np.argsort(-pro_y_pred_prob, axis=1)
two_pred = L[:,0:2]

class_dic = {mlp_clf.classes_[i]: i for i in range(len(mlp_clf.classes_))}
key_list = list(class_dic.keys()) 
val_list = list(class_dic.values()) 

dddd=[]
score = []
for i in range(len(pro_y)):
    first = two_pred[i][0]
    second = two_pred[i][1]
    label = list([key_list[val_list.index(first)],key_list[val_list.index(second)]])
    if y_test[i] in label :
        score.append(1)
    else : 
        score.append(0)
    dddd.append({'title':professor['title'][i],'y_test':professor['token'][i], 'first':label[0], 'second':label[1]})
#acc = sum(score)/len(y_test)
#print("2class 정확도:", acc)
ddd=pd.DataFrame(dddd)
ddd.head(20)
professor['predicted_class']='1:'+ddd['first']+' 2:'+ddd['second']

In [120]:
professor.head(10)

,title,press,link,date,content,token,class,predicted_class
0,"광운대 연구진, 다기능 나노 광학 디바이스 개발",매일경제,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-21 14:21,이상신 사진제공 이상신 광운대학교는 전자공학과 이상신 연구팀이 최덕용 호주국립대 ...,"[전자공학, 광학, 디바이스, 렌즈, 광학, 복잡, 구조, 가지, 광학, 소자, 극...",None,1:화학및섬유소재 2:지능형반도체
1,"이대목동병원 김건하 교수, 챗봇 회사에 ‘치매 예방 훈련용 콘텐츠’ 기술 이전",스포츠서울,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-18 10:26,김건하 교수스포츠서울김건하 이대목동병원 신경과 교수가 챗봇 회사인 주하이와 치매 예...,"[신경, 챗봇, 회사, 치매, 예방, 훈련, 콘텐츠, 협약, 체결, 치매, 예방, ...",None,1:홈어플라이언스 2:의료서비스·기기
2,"남덕우 UNIST 교수팀, 마이크로 RNA 조절 빅데이터 분석시스템 개발",전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-17 21:14,남덕우 UNIST 교수팀 왼쪽부터 김진환 연구원 남 윤소라 박사 하이 응우옌 박사U...,"[왼쪽, 인간, 마이크로RNA, 조절, 네트워크, 예측, 빅데이터, 분석, 개발했다...",None,1:의료서비스·기기 2:바이오
3,"박성규 GIST 교수, 혈액 한 방울로 치매 진단 기술 공동 개발",전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-17 12:02,국내 연구진이 혈액 한 방울로 치매를 간단히 진단할 수 있는 기술을 개발했다 치매 ...,"[혈액, 치매, 간단, 진단, 치매, 조기, 진단, 예방, 기여, 생명과학부, 의생...",None,1:바이오 2:의료서비스·기기
4,"선준브레인센터, 정지향 교수와 의료고문 및 연구 사업진행…선준미디어 교재개발",스포츠조선,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-15 13:06,사진 선준미디어 제공선준브레인센터는 대한신경과학회 치매 대책 특임이사이며 대한치매학...,"[치매, 신경, 정지, 의료, 치매안심센터, 치매환자, 참여, 계획, 미디어, 비교...",None,1:스마트미디어기기 2:조선
5,"서울백병원 김율리 교수팀, 'IT 기술+의료' 실시간 거식증 치료법 개발",스포츠조선,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-13 14:06,IT 기술과 의료가 접목돼 언제 어디서든 실시간으로 사용 가능한 거식증 치료법이 개...,"[IT, 의료, 접목, 어디서, 실시간, 거식증, 치료법, 거식증, 섭식장애, 심각...",None,1:홈어플라이언스 2:의료서비스·기기
6,"'자가진단 스마트빌딩 나온다' 김창석 부산대 교수, 광섬유센서 원천기술 개발",전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-13 14:05,실시간 준분포형 광섬유센서 기술을 개발한 김창석 부산대 교수부산대 연구팀이 스스로 ...,"[실시간, 광섬유센서, 스스로, 문제, 인지, 진단, 해결, 방안, 스마트, 스마트...",None,1:LED/광 2:정밀기계
7,"아주대 서형탁 연구팀, 수소 생성 '고효율 태양광촉매 전극' 개발",파이낸셜뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-12 10:04,구리복합산화물기반의‘물분해광전극’개발 아주대 서형탁 교수와 공동연구원 칼...,"[구리, 공동연구원, 신소재공학과, 고효율, 전극, 태양광, 분해, 수소, 전극, ...",None,1:에너지 2:화학및섬유소재
8,"포스텍 김원배교수팀, '온난화 주범' 이산화탄소 분해하는 촉매 성공",세계일보,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-12 3:03,포스텍연구팀이 지구온난화의 주범 이산화탄소를 분해해 산업적 활용가치가 높은 다...,"[지구온난화, 주범, 이산화탄소, 분해, 산업적, 활용가치, 물질, 전환, 촉매, ...",None,1:에너지 2:화학및섬유소재
9,"김용진 교수팀, 자궁근종 성장 예측 지표 개발",전자신문,https://news.naver.com/main/read.nhn?mode=LSD&...,2019-03-11 15:12,김용진 고대구로병원 산부인과 교수고려대학교 구로병원원장 한승규은 김용진 산부인과 교...,"[자궁근종, 예후, 예측, 지표, 성장, 양상, 치료, 필요, 경우, 경우, 그동안...",None,1:의료서비스·기기 2:바이오


In [121]:
professor.to_excel('C:/Users/hangy/Desktop/professor_predict_result.xlsx',sheet_name='Sheet1')